# APIM ❤️ OpenAI

## Built-in logging lab
![flow](../../images/built-in-logging.gif)

Playground to try the [built-in logging capabilities of API Management](https://learn.microsoft.com/azure/api-management/observability). The requests are logged into Application Insights, and it's easy to track request/response details and token usage with the provided [notebook](openai-usage-analysis-workbook.json).

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🔍 Analyze Application Insights requests](#kql)
- [🔍 Open the workbook in the Azure Portal](#portal)
- [🗑️ Clean up resources](#clean)

### Backlog
- Improve the notebook
- Use the [trace policy](https://learn.microsoft.com/azure/api-management/trace-policy) to send aditional info within the policy
- Add OpenAI diagnostics

### Prerequisites
- [Python 3.9 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [2]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

openai_resources = [
    {"name": "openai1", "location": "swedencentral"},
    {"name": "openai2", "location": "francecentral"}
]
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
# %load ../../shared/snippets/create-az-resource-group.py
# type: ignore

import datetime

resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print(f"✅ Azure Resource Group {resource_group_name} created ⌚ {datetime.datetime.now().time()}")


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# %load ../../shared/snippets/create-az-deployment.py
# type: ignore

import json

backend_id = "openai-backend-pool" if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_xml = policy_xml_file.read()

    if "{backend-id}" in policy_xml:
        policy_xml = policy_xml.replace("{backend-id}", backend_id)

    if "{aad-client-application-id}" in policy_xml:
        policy_xml = policy_xml.replace("{aad-client-application-id}", client_id)

    if "{aad-tenant-id}" in policy_xml:
        policy_xml = policy_xml.replace("{aad-tenant-id}", tenant_id)

    policy_xml_file.close()
open("policy-updated.xml", 'w').write(policy_xml)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"


<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [ ]:
# %load -r 1-21 ../../shared/snippets/deployment-outputs.py
# type: ignore

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = stdout.n
print(f"👉🏻 APIM Service Id: {apim_service_id}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = stdout.n
print(f"👉🏻 APIM Subscription Key (masked): ****{apim_subscription_key[-4:]}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = stdout.n
print(f"👉🏻 APIM API Gateway URL: {apim_resource_gateway_url}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = stdout.n
print(f"👉🏻 Workspace ID: {workspace_id}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = stdout.n
print(f"👉🏻 App ID: {app_id}")

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [ ]:
# %load ../../shared/snippets/api-http-requests.py
import json
import requests
import time

runs = 10
sleep_time_ms = 100
url = f"{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"
api_runs = []
messages = {"messages": [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

# Initialize a session for connection pooling
session = requests.Session()
# Set default headers
session.headers.update({'api-key': apim_subscription_key})

try:
    for i in range(runs):
        print(f"▶️ Run {i+1}/{runs}:")

        start_time = time.time()
        response = session.post(url, json = messages)
        response_time = time.time() - start_time
        print(f"⌚ {response_time:.2f} seconds")

        # Check the response status code and apply formatting
        if 200 <= response.status_code < 300:
            status_code_str = f"\x1b[1;32m{response.status_code} - {response.reason}\x1b[0m" # Bold and green
        elif response.status_code >= 400:
            status_code_str = f"\x1b[1;31m{response.status_code} - {response.reason}\x1b[0m" # Bold and red
        else:
            status_code_str = str(response.status_code)  # No formatting

        # Print the response status with the appropriate formatting
        print(f"Response status: {status_code_str}")
        print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")

        if "x-ms-region" in response.headers:
            print(f"x-ms-region: \x1b[1;32m{response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request
            api_runs.append((response_time, response.headers.get("x-ms-region")))

        if (response.status_code == 200):
            data = json.loads(response.text)
            print(f"Token usage: {json.dumps(dict(data.get("usage")), indent = 4)}\n")
            print(f"💬 {data.get("choices")[0].get("message").get("content")}\n")
        else:
            print(response.text)

        time.sleep(sleep_time_ms/1000)
finally:
    # Close the session to release the connection
    session.close()


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [ ]:
# %load ../../shared/snippets/openai-api-requests.py

import time
from openai import AzureOpenAI

runs = 10
sleep_time_ms = 100

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription_key,
    api_version = openai_api_version
)

messages = [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    start_time = time.time()
    response = client.chat.completions.create(model = openai_model_name, messages = messages) # type: ignore
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")
    if response.usage:
        print(f"Token usage: Total tokens: {response.usage.total_tokens} (Prompt tokens: {response.usage.prompt_tokens} & Completion tokens: {response.usage.completion_tokens})")
    print(f"💬 {response.choices[0].message.content}\n")

    time.sleep(sleep_time_ms/1000)


<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [ ]:
# type: ignore

import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = ! az monitor app-insights query --app {app_id} --analytics-query {query}
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.